In [116]:
import sys
import csv
import pandas as pd

if len(sys.argv)<5:
    print('result and gold not specified. Default is ./opencc/simp2multitrad_test_opencc_s2, gold is ./test_cases/simp2multitrad_test.csv')
    result='./opencc/simp2multitrad_test_opencc_s2t'
    gold='./test_cases/simp2multitrad_test_gold.csv'
else:
    result=sys.argv[1]
    gold=sys.argv[2]
    print (result,gold)



result and gold not specified. Default is ./opencc/simp2multitrad_test_opencc_s2, gold is ./test_cases/simp2multitrad_test.csv


In [117]:
gold_csv = pd.read_csv(gold)


In [118]:


line_num=0
num_incor=0
num_corr=0
print ('printing incorrect predictions')
print ('char_gold','char_res')
with open(result, 'r') as res_f:
    
    for line in res_f:
        char_i=gold_csv['char_index'][line_num]
        char_gold=gold_csv['gold_char'][line_num]
        char_res=line[char_i]
        

        if char_gold!=char_res:
            print (char_gold,char_res,char_i,line_num,gold_csv['gold'][line_num])
            num_incor+=1
        else:
            num_corr+=1
        line_num+=1   
print (num_incor)
print (float(num_incor/len(gold_csv)), 'incorrect. total 133000',)

printing incorrect predictions
char_gold char_res
台 臺 5 0 王清峰批評台電瞞天過海偷渡核四決標
台 臺 0 1 台灣省各界慶祝中華民國八十五年母親節表揚模範母親大會,明天下午三時三十分,在台灣省政府台中黎明辦公區中正堂舉行,台灣省長宋楚瑜將在會中頒贈「明倫教孝」匾額給三十七位模範母親。
台 臺 13 2 馬拉杜納在動身前夕接受電視台訪問時,發表感人談話,他說:「我深愛你們,我會盡全力活下去,我仍與你們同在,未來將如何,應由我而非他人決定;我從內心深愛著你們。
台 臺 8 3 中央社記者黃慧敏台北二十日電
台 臺 10 4 國際美元持續疲軟,新台幣兌美元今天再度走升,由於外資匯入,新台幣兌美元開盤後升值幅度一路擴大,九時三十分以後突破32.2元,最高來到32.175元,升值1.35角;九時四十五分新台幣兌美元匯率為32.189元,升值1.21角,台北外匯經紀公司成交量1.4億美元。
台 臺 9 5 (中央社記者管中維台北二十七日電)
台 臺 11 6 WTO漁業補貼談判  台日韓提案強調透明性
台 臺 3 7 他說,台灣的戰略地位重要。
台 臺 6 8 劉泰英評估,台灣只有兩家航空公司體系運作的空間,因此中華開發如能取得華航的經營權,加上目前已主導復興航空經營,並為遠東航空大股東的優勢,將在復興、遠航兩家航空公司合併後,再與華航進行合併,成為台灣兩大航空體系之一。
台 臺 7 9 砂眼防治計畫是台灣參與聯合國相關援助計畫的範例計畫,執行方式是協助布國居民及兒童種植蔬果,補充維他命A ,加強醫療設施及教育衛生習慣以預防砂眼。
台 臺 3 10 中央社台北15日電 09/15/09  12:47:06
台 臺 7 11 中央社記者方旭台北二十二日電
台 臺 4 12 他強調,台灣職棒的明天,繫乎究竟有多少選手能嚴拒黑金的威脅和利誘。
台 臺 0 13 台股今天在法人作帳行情推升下,指數開高走高站上7800點,並以今年最高點7823.72點封關。
台 臺 4 14 威力彩新台幣11.2億元得主到今天上午9時未現身,不少台東民眾戲稱「就是我啦!
台 臺 0 15 台股昨天衝高挑戰7900點關卡,尾盤未能站上,僅以平盤作收,台指期小漲,逆價差縮小到35.99點,美股四大指數清晨收盤漲多跌少。
台 臺 1 16 來台之前,翟

儘 盡 44 2939 對海協會也有意籌組文教參訪團訪台一事,海基會在函中說,在海基會文教參訪團赴大陸參訪後,會儘速安排。
儘 盡 6 2943 他也敦促國會儘快通過包括病人權利、槍枝管制、防止仇恨犯罪及改善學校教育等其他重要法案。
儘 盡 12 2944 葉爾勤指車臣戰事已結束將儘快重建
儘 盡 5 2947 同時,還將儘快制定相應的法規、政策,依法對江河源頭地區的生態環境進行保護和治理。
儘 盡 94 2949 談到電源的運用時,總統說,就現況與未來發展而言,電源的進一步開發,是必然的方向;總統表示,過去政府在開發電源的政策上,是有不夠週延的評估,為了避免再發生電源不足,或是必須限電的不便,政府將儘量促使改進,以協助解決電源不足產生的問題。
儘 盡 71 2960 「扁宋會」今天達成10點共識,包括加強推動兩岸經貿、文化與學術交流,逐步推動貨運便捷化,乃至全面三通;對影響產業發展及國家整體競爭力的管制,將儘速予以檢討和修正,以落實生根台灣、布局全球的經發會共識。
儘 盡 88 2983 針對媒體報導中共擬推動兩岸記者常駐,行政院大陸委員會副主委鄧振中今天表示,這再度證明兩岸應該擱置爭議,先針對有迫切性的議題商談,但這必須回到體制內、制度化管道中來談,由兩岸兩會儘早展開商談。
儘 盡 81 2986 藝人崔愛蓮三日因為氣喘病發作病逝,與楊耀東結褵多年鶼鰈情深,對於愛妻猝逝,楊耀東至今仍傷痛不已無法接受,而崔愛蓮年邁雙親今天都到醫院看愛女最後一面,姐姐崔苔菁也將儘速返台料理後事。
儘 盡 16 2990 在被問到完成慰留後,總統府是否會儘快完成陳履安的辭職手續?
儘 盡 57 2991 彭文正指出,目前省委會輔選部門正在研定有關鄉鎮市長與民意代表選舉的作業流程,所以一切的事情尚未成熟,但是提名時間將儘量配合政府的時程,而且因為鄉鎮市長與民代選舉的性質與其他選舉不同,國民黨將不會太早公佈提名人名單。
松 鬆 14 3007 檢方十一日依貪污治罪條例將洪松茂聲請羈押禁見獲准,並諭令富邦產險經理柯麗雪新台幣五十萬元交保。
松 鬆 121 3010 南投縣阿扁之友會下午在草屯鎮中山公園舉行成立大會,而草屯鎮公所也同時在中山公園舉辦農特產品展售會,使得會場熱鬧滾滾,下午三時陳水扁進入會場即受到民眾的熱烈歡迎,「總統好」的口號聲不斷,陳水扁先授旗給南投縣阿扁之友會會長洪瑞明及授

郁 鬱 0 5400 郁慕明表示,新黨雖然是小黨,但是如水般清純如一,是在野聯盟中不可或缺的要素,尤其沒有黑金色彩,但只是內部紛爭,讓新黨由盛而衰,因此做為創黨人之一,感到相當慚愧與痛心,所以今天起而行,希望所有愛新黨的人能團結,讓新黨走出陰影,跨過百分之五門檻。
郁 鬱 1 5401 蔡郁璇自嘲說,之前掉髮的情形,很像她的偶像李嘉欣多年前演的港片「豬扒大聯盟」劇中,飾演患有地中海禿頭的職場女性。
郁 鬱 4 5402 立法委員郁慕明今天下午籲請交通部考量船商經營成本,開放民間航商經營海峽兩岸的定期航線業務。
郁 鬱 25 5403 台北縣卸任鄉鎮市長中,有泰山鄉長黃中興、五股鄉長蔡郁男、蘆洲市長周慶陽轉任縣議員,因此,配合他們上午在縣議會宣誓就職,這三個鄉鎮長交接在下午進行。
郁 鬱 22 5404 周光宙表示,去年10月13日,他就與新黨主席郁慕明、台北市建築師公會理事長練福星等人聯袂前往大陸拜會有關部門,積極商談如何落實取得大陸註冊建築師資格辦理一次性評估認定相關事宜,並取得明確進展。
郁 鬱 5 5406 油畫類:林郁廷、彭昭如(獎金新台幣五萬元)。
郁 鬱 9 5408 第一名–泰山國小方郁集,可獲獎金新臺幣一千二百元及獎狀;第二名–明志國小張睿欣、板橋國小許嘉珍,二人可各獲獎金一千元及獎狀;第三名–修德國小陳明明、中正國小朱宏文、大豐國小張永欣,三人各獲獎金八百元及獎狀。
郁 鬱 46 5411 佔地五十公頃的歷史博物館,今天在安南區和順寮舉行開工動土典禮,主體建物有兩棟,由文建會主委林郁秀主持,包括立法委員賴清德、王昱婷、錢林慧君、唐碧娥、市議員李文正、郭信良、文化局長許耿修、都發局長李得全等人,在動土典禮後,所有人並在印有台灣模型的陶版上留下姓名,留下歷史性的一刻。
郁 鬱 30 5412 這次入圍前五名作品包括有楊甯「純律」、成英姝的「地獄門」、林郁庭的「離魂香」、康祁「將薰」及梁亨的「最美的東西」。
郁 鬱 1 5413 黃郁文連續兩次未出席市議會定期大會,面臨被撤銷議員及議長資格的命運。
郁 鬱 28 5415 至於馬總統下午親自澄清綠營人士指政府下令撤除國旗一事,王郁琦表示,總統在輿情小組會議中肯定警方執行維安作業「保障合法、取締非法」的辛勞,之所以親自出面澄清,是因總統根本沒有下令撤國旗,無法接受綠營人士的扭曲。
郁 鬱 17 5418 其

划 劃 42 7857 元智大學推動燃料電池做為替代能原不遺餘力,今天在新春記者會展示新近組裝完成的燃料電池划板車,由於燃料電池來自外國,成本高達新台幣十萬元,有待元智大學燃料研究中心研發突破,才可望進一步產製燃料電池摩托車,展示的燃料電池教具成本不到外國產品三成,極具市場競爭力。
划 劃 69 7860 金門二零零六金湖鎮海灘花蛤季,今天在南海岸成功海灘展開,湧進上千民眾參加,活動代言人藝人胡晴雯偕同福建省主席顏忠誠、金湖鎮長李成義客串表演划旱船,揭開挖花蛤、牽罟、搖滾晚會、花蛤仙子選拔等熱情有勁的活動序幕。
划 劃 37 7862 片中主角是六十八歲的夏樸馬卡拉修(音譯)搭著雅美族傳統小舟在黎明時出海,邊划邊唱歌給魚聽之後,鬼頭刀魚就躍出海面的情形。
划 劃 16 7865 劉成的屍體是去年11月1日被兩名划獨木舟的兒童在雪梨南方的喬治河(Georges River)發現。
划 劃 64 7866 活動將於二十三日在官田鄉舉行,內容包括寫生比賽、賞菱之旅、白鷺歸巢、傳統民俗技藝展、菱角生態及參觀國寶級菱鳥水雉保護區、採紅菱、划菱舟比賽、菱角美食大餐等,現場並將邀請官田藝術家陳泰源展出作品,也有陶瓷手拉坯現場製作、國立台南藝術學院國樂演奏等。
划 劃 25 7871 此外,會議將成立技術小組,正式啟動「渤海碧海行動計划」。
涂 塗 23 7972 民進黨屏東縣黨部辦理立委黨內初選登記之初,雖有涂順振抗議事件,但最後涂順振退出參與初選,因此登記者只有現任立委邱連輝和增額國代蘇嘉全兩人。
涂 塗 3 7973 副議長涂銓重及議員們初步了解,粘仲仁將被移送桃園地檢署偵辦,今天下午將趕往桃園了解。
涂 塗 47 7974 第三屆嘉義市應選名額是一人,國民黨原有吳育昇、陳適庸、陳鏡仁三人參加提名登記,後兩人稍後退出;涂德錡表示,如果蕭萬長確定參選,黨將會為吳育昇安排出路。
涂 塗 14 7975 女子組比賽,我國旅居日本名將涂阿玉一枝獨秀,以二一六桿獲得第一名,獨得獎金八萬元,亞軍黃玥(玨八)二二五桿,獎金五萬四千元,吳明月以二二六桿獲第三名。
涂 塗 0 7976 涂太太今天下午搭乘國泰班機至香港後,再轉機飛往德國,被問到近況時,她表示,想引進德國畫家的作品到台灣展覽,屆時她會待在台灣的時間就會比較長一些。
涂 塗 0 7977 涂美瑜提醒民眾要把未食用完的粽子放進冷凍庫

彌 瀰 77 8712 旅社服務生見兩人應於今天中午十二時退房,卻不見人影,上樓叫門也無人應門,十分可疑,於是破門而入查看究竟,當場發現兩人倒臥在房內,現場門窗緊閉,燃燒木炭煙霧彌漫,且現場炭火仍未熄滅,於是緊急將他們送高雄醫學大學附設醫院急救,兩人經急救後,已脫離險境,分別被雙方家屬轉院至台南市及台南縣的醫院,進一步急救。
彌 瀰 47 8764 龍應台則表示,五四運動距今八十八年,解嚴也有十九年,反對者成為執政者,掌權者成為在野黨,但社會彌漫猜忌與不信任,人民對現狀不滿的憤怒被無力感取代。
斗 鬥 91 8827 第二局比賽,片井更有如進入無人之地,先是三連倒,之後在一個半倒後連續六連倒,曾素芬包括三連倒在內共有四次全倒,結果片井以五百分對四百零二分的比數奪得女盟主賽金牌,曾素芬得到銀牌,南芝斗獲得銅牌。
斗 鬥 20 8842 台灣籍商船天德輪今天下午三時許,在行經目斗嶼西北西方五十海里處海域,因澎湖海域風浪甚大,而遭強浪侵襲引擎進水停車沈沒,十名船員由海鷗部隊直昇機救起,載回馬公機場。
斗 鬥 16 8866 以前的人說:「一府二鹿三艋舺四寶斗」,也有人說「寶斗有三圓」:教員、警員與肉圓,就是說北斗子弟當老師、當警察的特別多。
斗 鬥 22 8882 中共「新華社」自西安報導,在延川縣馬家河鄉用斗村出土的青銅器,是屬商代中期至晚期遺物。
斗 鬥 32 8897 設在二林鎮東興社區活動中心的第六百二十投開票所,以及設在二林鎮原斗國小教室的第六百三十六投開票所,今天上午都發生投票所錄影引起選民不滿的打架糾紛,已由駐區檢察官指示二林警察分局偵辦,二林鎮公所曾以電話,向彰化縣選舉委員會請示,而中央選委會秘書長許桂霖請台灣省選委會轉達選務機關不能主動錄影的指示。
蔑 衊 85 8908 黃榮華今天駁回這項會議召開申請,他說,信用合作社選舉罷免法是程序法,除了在流程上有一定的規範,所附證據與罷免理由書必須有相當關聯,因此,如果證據與罷免書所指陳的不相干、污蔑或誣告,即可以程序不合駁回。
蔑 衊 27 8910 他說,由此可看出,檢調人員在辦案時經常會遭到不法份子污蔑、恐嚇、抹黑,無所不用其極,連他自己也經常遭到污蔑、抹黑,有某雜誌就經常在社論上罵他,但是法務部仍然堅持查賄工作。
蔑 衊 73 8911 朱立倫表示,他在近半年來就聽到立法院與坊間都流傳他有緋聞傳聞,而且傳

伙 夥 17 9464 馬總統指出,台灣也與日本建立「特別伙伴關係」,在日本札幌設立辦事處,10月底也將實現台北松山機場與東京羽田機場對飛,爭取到英國、愛爾蘭、紐西蘭、斐濟等國家免簽證待遇,並獲歐洲議會多次通過決議支持。
伙 夥 29 9465 --十八時三十分,前國民黨主席馬英九出席「阿九仔和黑手ㄟ做伙搏感情」餐會台電勵進餐廳,和平東路一段七十五巷內。
伙 夥 73 9466 南韓青瓦台當局今天發表評論指出,南韓政府真誠地歡迎美國共和黨候選人布希總統的蟬聯勝選,期盼以布希總統的蟬聯為契機,更加強發展「韓美兩國總體且躍動的伙伴關係」。
咸 鹹 28 9470 嗣膺任國民大會秘書長,宣力民主,淵謨深運;調和鼎鼐,朝野咸欽。
咸 鹹 5 9473 其次,陳文咸指出, 8吋廠技術已不是「機密」,即使台灣的 8吋廠不登陸,其他國家的 8吋廠也會競相到大陸投資設廠,屆時台灣反而失去了好機會,所以TSIA希望政府不要管太多,而能讓台灣的8吋廠到大陸投資設廠。
咸 鹹 1 9474 麥咸市副市長歧視華人言論風波愈演愈烈
咸 鹹 9 9479 台北市文化局局長廖咸浩率領的參訪團今天抵達北京,開始兩天的交流參訪活動。
咸 鹹 1 9481 廖咸浩表示,兩岸文化交流近年來持續熱絡,相互間瞭解不斷深入並拓展到很多新領域,台北市文化局願意在這方面做更多的工作。
咸 鹹 1 9484 廖咸浩說,蔡瑞月一生命運坎坷,與外省知識青年雷石榆婚姻美滿,卻遭政治迫害拆散,兩人兩岸乖隔,她也琅璫入獄,雖飽受挫折,因情懷雷石榆,始終不屈不撓創作,舞蹈社平凡的木造建築,因努力耕耘孕育文化光彩,市府於民國八十八年指定為市定古蹟。
咸 鹹 5 9487 文化局長廖咸浩表示,大樹分布密度不高,除了多種樹改善,調查或有疏漏,歡迎民眾補遺,提升都市大樹密度。
咸 鹹 5 9493 文化局長廖咸浩指出,今年開始,依新版文資法,鑑定登錄歷史建築,一併確認歷史意涵與保存範圍,例如是否保存立面、騎樓、特殊構件或全棟保存等。
咸 鹹 0 9495 咸認阿比歐拉是去年六月十二日總統大選的獲勝者,但是當時的軍事領袖巴班吉達以選舉發生弊端為由,宣布投票結果無效。
咸 鹹 17 9496 每日郵報報導說,英國超級聯賽球會富咸已與日本球會大阪松下簽署借員協議,將借用日本足球明星稻本潤一一年。
咸 鹹 7 9498 駐新加坡大使:咸明澈,

饑 飢 55 11417 至於交易時間恢復為舊制的上午9時至12時或12時30分,魏哲禎指出,現制於下午1時30分收盤,讓營業員必須忍受饑餓上班,且壓縮下午拜訪客戶、拓展業務的時間;對投資人而言,午餐時間已到但尚未收盤,去用餐也不是,留著看盤又得忍受饑餓,都是折磨。
饑 飢 22 11418 」、「(忽視農民工性需求)這是飽漢子不知餓漢饑!
饑 飢 13 11419 動畫電影「王子與公主」、「饑哩咕與女巫」、「凱薩琳、麗克維絲專輯」、「仕女與士兵」、「聖彼得堡聖女」;夏布洛電影「好男賽吉」、「雙輪」、「該死的人」、「帽屋幽靈」、「探長拉瓦丁」、「貝蒂」、「儀式」、「一切搞定」、「謝謝妳的巧克力」、「養子」。
饑 飢 31 11420 」金大中在提及前天遭中共遣返的七名北韓人時說:「我一想到這些因饑餓而脫逃的 (北韓) 人在遭遣返後將面臨更多苦難,就會深感愧疚。
饑 飢 14 11423 嘗過風災滋味的美國,將秉持人饑己饑精神更積極參與國際事務,抑或獨善其身?
饑 飢 28 11424 縣總工會表示,希望所轄各產業工會成員能秉持「人溺己溺、人饑己饑」精神,共同來伸援受災戶渡過難關,早日重建家園。
饑 飢 42 11427 國民黨新竹縣委員會同時籲請公司行號企業界、社團、寺廟及各界善心人士,發揮人溺己溺、人饑己饑的同胞愛精神,踴躍捐輸現金或物資,讓災區民眾渡過難關。
饑 飢 14 11429 行政院強調,感謝各界發揮「人饑己饑、人溺己溺」的精神,熱心捐款幫助災民,全部捐款及上網資料將會交由「財團法人九二一賑災重建基金會」妥善運用。
饑 飢 23 11431 –全球三十個彼此間有衝突的國家將利用糧食戰爭和饑餓作為一種武器。
饑 飢 7 11432 身為第十三屆「饑餓三十」人道救援代言人的阿雅和黑人,日前才從馬拉威返回台灣,今天下午出席在台北西門町舉行的「愛心上線,串聯希望」活動,並分享馬拉威之行的心得。
饑 飢 9 11434 高雄民間機構秉持人饑己饑、人溺己溺精神,再度於今年舉辦「愛讓我們更好」愛心義賣cd活動,期盼透過義賣與捐款讓失親兒童也能在寒冬感受到社會溫馨。
饑 飢 68 11439 吉田富沿郎表示,「世界和平鐘之會」設立宗旨是根源於聯合國憲章的世界和平運動,以「世界和平鐘」為中心,不分男女、宗教、種族,共同掃除戰爭、饑餓、不幸與恐怖,喚醒人類追求和平。
饑 飢